In [21]:
from pyspark.sql import SparkSession

In [22]:
spark =  SparkSession\
        .builder\
        .appName('readfiles')\
        .getOrCreate()
spark        

In [93]:
## read csv file
df = spark.read.csv("train_schedule.csv", header=True)
df.show()

+--------+-------------+-----+
|train_id|      station| time|
+--------+-------------+-----+
|     324|San Francisco|7:59a|
|     324|  22nd Street|8:03a|
|     324|     Millbrae|8:16a|
|     324|    Hillsdale|8:24a|
|     324| Redwood City|8:31a|
|     324|    Palo Alto|8:37a|
|     324|     San Jose|9:05a|
|     217|       Gilroy|6:06a|
|     217|   San Martin|6:15a|
|     217|  Morgan Hill|6:21a|
|     217| Blossom Hill|6:36a|
|     217|      Capitol|6:42a|
|     217|       Tamien|6:50a|
|     217|     San Jose|6:59a|
+--------+-------------+-----+



In [51]:
# Another way of reading same file
df = spark.read.option("header","True").csv("train_schedule.csv")
df.show()

+--------+-------------+-----+
|train_id|      station| time|
+--------+-------------+-----+
|     324|San Francisco|7:59a|
|     324|  22nd Street|8:03a|
|     324|     Millbrae|8:16a|
|     324|    Hillsdale|8:24a|
|     324| Redwood City|8:31a|
|     324|    Palo Alto|8:37a|
|     324|     San Jose|9:05a|
|     217|       Gilroy|6:06a|
|     217|   San Martin|6:15a|
|     217|  Morgan Hill|6:21a|
|     217| Blossom Hill|6:36a|
|     217|      Capitol|6:42a|
|     217|       Tamien|6:50a|
|     217|     San Jose|6:59a|
+--------+-------------+-----+



### Spark provides this functionality of not to define schema and tries to infer it automatically

In [52]:
df = spark.read.csv("train_schedule.csv", header = True, inferSchema = True)
df.printSchema()

root
 |-- train_id: integer (nullable = true)
 |-- station: string (nullable = true)
 |-- time: string (nullable = true)



### StructType and StructField combination help you to define your own schema

In [100]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, StringType, TimestampType

### Create your own schema by referring columns and casting it to required type

In [99]:
data_schema = StructType([
                            StructField('train_id', IntegerType(), True),
                            StructField('station', StringType(), True),
                            StructField('time', TimestampType(), True)
])

### Read file by referring your custom schema

In [68]:
df = spark.read.csv("train_schedule.csv", schema = data_schema)

In [69]:
df.printSchema()

root
 |-- train_id: integer (nullable = true)
 |-- station: string (nullable = true)
 |-- time: timestamp (nullable = true)



### Convert Spark dataframe to Pandas dataframe using toPandas()

In [108]:
df = spark.read.csv("train_schedule.csv", header = True)
type(df)

pyspark.sql.dataframe.DataFrame

In [107]:
## read spark dataframe
df.head(2)

[Row(train_id='324', station='San Francisco', time='7:59a'),
 Row(train_id='324', station='22nd Street', time='8:03a')]

In [104]:
pdDF = df.toPandas()
type(pdDF)

pandas.core.frame.DataFrame

In [96]:
#read pandas dataframe
pdDF.head(2)

,train_id,station,time
0,324,San Francisco,7:59a
1,324,22nd Street,8:03a


### Convert Pandas dataframe to Spark dataframe using spark.createDataFrame()

In [110]:
sparkDF = spark.createDataFrame(pdDF)
type(sparkDF)

pyspark.sql.dataframe.DataFrame

### Convert RDD to Spark Dataframe

In [119]:
from pyspark import SparkConf
from pyspark import SparkContext
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

In [122]:
rddSc = sc.parallelize([1,2,3])
rddSc

ParallelCollectionRDD[312] at parallelize at PythonRDD.scala:195

### Executing SQL statment on File by registering it as tables

In [127]:
spark.catalog.listTables()

[]

In [4]:
data = spark.read.csv('train_schedule.csv', header=True)

In [5]:
data.createOrReplaceTempView('data_table')

In [138]:
query = 'SELECT * FROM data_table'
spark.sql(query).show(5)

+--------+-------------+-----+
|train_id|      station| time|
+--------+-------------+-----+
|     324|San Francisco|7:59a|
|     324|  22nd Street|8:03a|
|     324|     Millbrae|8:16a|
|     324|    Hillsdale|8:24a|
|     324| Redwood City|8:31a|
+--------+-------------+-----+
only showing top 5 rows



### Spark SQL 6 : Chek tables presence using catalog.listTables()

In [ ]:
# temporary table is created using createOrReplaceTempView
data_table = data.createOrReplaceTempView('data_table')

In [141]:
spark.catalog.listTables()

[Table(name='data_table', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [143]:
# drop your table
spark.catalog.dropTempView('data_table')

In [144]:
# check again
spark.catalog.listTables()

[]

## SparkSQL 7

### Caching and UnCaching tables

#### To cache Table/View : spark.catalog.cacheTable()
#### To uncache Table/view : spark.catalog.uncacheTable()
#### To check wheter cached or not : spark.catalog.isCached()

In [6]:
# List the tables
print("Tables:\n", spark.catalog.listTables())

# Cache table1 and Confirm that it is cached
spark.catalog.cacheTable('data_table')
print("table1 is cached: ", spark.catalog.isCached('data_table'))

# Uncache table1 and confirm that it is uncached
spark.catalog.uncacheTable('data_table')
print("table1 is cached: ", spark.catalog.isCached('data_table'))

Tables:
 [Table(name='data_table', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]
table1 is cached:  True
table1 is cached:  False


## #SparkSQL 8

### Caching Spark DataFrames
#### To cache : cache() or persist()
#### To uncache : unpersist()

In [7]:
data = spark.read.csv('train_schedule.csv', header=True)

In [18]:
data.cache()
data.is_cached

True

In [20]:
data.unpersist()
data.is_cached

False

In [26]:
data.persist(storageLevel= pyspark.StorageLevel.MEMORY_AND_DISK)
data.is_cached

True

## SparkSQL 9

#### Query data using DataFrame API and Spark SQL

In [28]:
data.show(5)

+--------+-------------+-----+
|train_id|      station| time|
+--------+-------------+-----+
|     324|San Francisco|7:59a|
|     324|  22nd Street|8:03a|
|     324|     Millbrae|8:16a|
|     324|    Hillsdale|8:24a|
|     324| Redwood City|8:31a|
+--------+-------------+-----+
only showing top 5 rows



In [27]:
data.groupBy('train_id').agg({'station':'count'}).show()

+--------+--------------+
|train_id|count(station)|
+--------+--------------+
|     217|             7|
|     324|             7|
+--------+--------------+



In [31]:
data.createOrReplaceTempView('data_table')
query ='SELECT train_id, count(station) FROM data_table GROUP BY train_id'
spark.sql(query).show()

+--------+--------------+
|train_id|count(station)|
+--------+--------------+
|     217|             7|
|     324|             7|
+--------+--------------+



## Spark SQL 10

### GlobalTempView

Temporary views in Spark SQL are session-scoped and will disappear<br> 
if the session that creates it terminates. If you want to have a temporary view<br>
that is shared among all sessions and keep alive until the Spark application terminates,<br>
you can create a global temporary view.

* Don't forget to include **global_temp** while referring global table/view

In [ ]:
data.createGlobalTempView('dataglb_tbl')

In [34]:
query = 'SELECT * FROM global_temp.dataglb_tbl'
spark.sql(query).show(5)

+--------+-------------+-----+
|train_id|      station| time|
+--------+-------------+-----+
|     324|San Francisco|7:59a|
|     324|  22nd Street|8:03a|
|     324|     Millbrae|8:16a|
|     324|    Hillsdale|8:24a|
|     324| Redwood City|8:31a|
+--------+-------------+-----+
only showing top 5 rows



## Spark SQL 11

A common confusion happens while creating dataframe in Pandas and Spark.
* To create dataframe in pandas use: DataFrame()
* To create data dataframe in spark : createDataFrame()

In [ ]:
pd_data = pd.DataFrame( 
                        {'Name' : 'Sumo', 'City':'UK'},\
                        {'Name' : 'Mark', 'City':'US'}
                    )
print('******* Pandas DatFrame*******')
print(pd_data.head())

print('******* Spark DatFrame*******')
spark_data= spark.createDataFrame(pd_data, schema=['Name', 'City'])
print(spark_data.show())

## SparkSQL 12

Format : <b> createDataFrame(data, schema=None, verifySchema=True)</b>

Spark DataFrame can only be created using <br>
<b>RDD, a list or a pandas.DataFrame.</b><br>

When schema is None, it will try to infer the schema<br>
(column names and types) from data, which should be 
<b>an RDD of Row, or namedtuple, or dict</b>

In [36]:
from pyspark.sql import Row
from pyspark.sql.types import StringType, StructType, StructField

#define schema, if you don't pass it then spark will try
#to infer it automatically
schema = StructType([
                    StructField('Name' , StringType(), True),
                    StructField('City', StringType(), True)
                ])

spark_data = spark.createDataFrame([Row('Sumo','UK'),
                                    Row('Mark','US')], schema=schema)
#print(spark_data.head())

## Spark SQL 13

### Changing partitions of Spark DataFrame
* To check the existing partition : getNumPartitions()
* To change partition : repartition()

In [57]:
df = spark.read.csv("train_schedule.csv", header = True)
print(type(df))
df.select('train_id').distinct().show()

<class 'pyspark.sql.dataframe.DataFrame'>
+--------+
|train_id|
+--------+
|     217|
|     324|
+--------+



In [58]:
print("The dataframe has partition:", df.rdd.getNumPartitions())
# Chagnging partition to 2
df = df.repartition(2, 'train_id')
print("The dataframe has new partition:", df.rdd.getNumPartitions())

The dataframe has partition: 1
The dataframe has new partition: 2


### Explain plan dataframes and tables

### UDF functions